In [8]:
import pandas as pd
import psycopg2

In [9]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port=5432
)

In [12]:
with conn:
    with conn.cursor() as cur:
        query = """
CREATE TABLE movies (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255),
  year DATE,
  description TEXT
);

CREATE TABLE genres (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255)
);

CREATE TABLE countries (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255)
);

CREATE TABLE staff (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255),
  role VARCHAR(255)
);

create table Movies_Genres (
  movie_id int references movies (id),
  genre_id int references genres (id),
  primary key (movie_id, genre_id)
);

create table Movies_Countries (
  movie_id int references movies (id),
  country_id int references countries (id),
  primary key (movie_id, country_id)
);

create table Movies_Staff (
  movie_id int references movies (id),
  staff_id int references staff (id),
  primary key (movie_id, staff_id)
);

CREATE TABLE users (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255),
  email VARCHAR(255),
  login VARCHAR(255),
  password VARCHAR(255)
);

CREATE TABLE logs (
  id SERIAL PRIMARY KEY,
  user_id INT,
  datetime_ TIMESTAMP,
  duration INT,
  movie_id INT,
  FOREIGN KEY (user_id) REFERENCES users (id),
  FOREIGN KEY (movie_id) REFERENCES movies (id)
);

CREATE TABLE recs (
    id SERIAL PRIMARY KEY,
    movie_id int references movies (id),
    user_id int references users (id)
);

CREATE TABLE sub_recs (
    rec_id int references recs (id),
    movie_id int references movies (id),
    primary key (rec_id, movie_id)
);"""
        cur.execute(query)
        conn.commit()

In [13]:
df_movies = pd.read_csv("train/movies.csv")
df_movies.head()

,id,name,year,date_publication,description,genres,countries,staff
0,0,"Мама, я дома",2022-01-01,2022-11-23T00:00:00,Где-то в глубинке вместе с дочерью и внуком жи...,[97],[238],"[1883, 33655, 25890, 1001, 12051, 10110, 16895]"
1,1,Три метра над уровнем неба,2010-01-01,NaN,"История любви парня и девушки, принадлежащих к...","[138, 97, 294]",[242],"[18168, 23444, 10850, 21847, 30555, 24469, 268..."
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,NaN,"690 год нашей эры, Китай. Первая императрица д...","[294, 302]","[250, 117]","[3933, 19953, 32174, 30640, 14127, 32141, 2549..."
3,3,Капитан,2017-01-01,2022-10-20T00:00:00,Вторая мировая война подходит к концу. В это в...,"[97, 303, 143, 319]","[188, 212, 0]","[16006, 12217, 30668, 28806, 16172, 5045, 1663..."
4,4,Лазурный берег,2015-01-01,2023-01-12T00:00:00,У бывшей танцовщицы Ванессы и писателя Роланда...,"[138, 97]","[1, 102, 0]","[23586, 8823, 8040, 34555, 32484, 18446]"


In [14]:
df_movies.drop(columns=["date_publication"], inplace=True)
df_movies.head()

,id,name,year,description,genres,countries,staff
0,0,"Мама, я дома",2022-01-01,Где-то в глубинке вместе с дочерью и внуком жи...,[97],[238],"[1883, 33655, 25890, 1001, 12051, 10110, 16895]"
1,1,Три метра над уровнем неба,2010-01-01,"История любви парня и девушки, принадлежащих к...","[138, 97, 294]",[242],"[18168, 23444, 10850, 21847, 30555, 24469, 268..."
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,"690 год нашей эры, Китай. Первая императрица д...","[294, 302]","[250, 117]","[3933, 19953, 32174, 30640, 14127, 32141, 2549..."
3,3,Капитан,2017-01-01,Вторая мировая война подходит к концу. В это в...,"[97, 303, 143, 319]","[188, 212, 0]","[16006, 12217, 30668, 28806, 16172, 5045, 1663..."
4,4,Лазурный берег,2015-01-01,У бывшей танцовщицы Ванессы и писателя Роланда...,"[138, 97]","[1, 102, 0]","[23586, 8823, 8040, 34555, 32484, 18446]"


In [15]:
df_movies['year'] = pd.to_datetime(df_movies['year'])

In [18]:
df_movies["description"].fillna("Отсутствует", inplace=True)
df_movies[df_movies["description"] == "Отсутствует"].head(10)

,id,name,year,description,genres,countries,staff
7,7,FOMENKO_ПРОГРАММЫ Серия 1,2018-01-01,Отсутствует,[320],[238],[]
80,80,Клеопатра,1934-01-01,Отсутствует,"[97, 124, 319]",[102],"[3866, 19325, 14169, 9955, 30969, 13612]"
106,106,Общество мертвых поэтов,1989-01-01,Отсутствует,"[97, 46]",[102],"[32694, 18829, 31557, 5730, 32271, 35560]"
113,113,Крепись!,2014-01-01,Отсутствует,"[46, 100]",[102],"[23780, 26541, 18158, 19972, 2048, 32889, 3438..."
142,142,Концерт Стаса Старовойтова,2016-01-01,Отсутствует,"[309, 362]",[238],[29124]
222,222,Морские котики,2017-01-01,Отсутствует,[],[],[]
421,421,Возмещение ущерба,2001-01-01,Отсутствует,"[97, 303, 294]","[49, 102]","[17310, 3823, 12856, 32406, 6480, 34880, 34135..."
444,444,DC девчонки-супергерои: Межгалактические игры,2017-01-01,Отсутствует,"[302, 261, 130]",[102],"[25277, 30213, 16147, 4462, 10032, 29747, 2278..."
471,471,Смурфики: Легенда о Смурфной лощине,2013-01-01,Отсутствует,[304],[102],"[7545, 7677, 17207, 30233, 14441]"
478,478,Пучина,1958-01-01,Отсутствует,[97],[205],[]


In [19]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO movies (id, name, year, description) VALUES (%s, %s, %s, %s)"
        for _, row in df_movies.iterrows():
            cur.execute(query, (row["id"], row["name"], row["year"], row["description"]))
        conn.commit()

In [20]:
import ast

df_movies["genres"] = df_movies["genres"].map(ast.literal_eval)

In [21]:
df_genres = pd.read_csv("train/genres.csv")
df_genres.head()

,id,name
0,2,Сказка
1,8,Здоровье
2,17,Наука
3,24,Комедийная мелодрама
4,38,Мистика


In [23]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO genres (id, name) VALUES (%s, %s)"
        for _, row in df_genres.iterrows():
            cur.execute(query, (row["id"], row["name"]))
        conn.commit()

In [24]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO Movies_Genres (movie_id, genre_id) VALUES (%s, %s)"
        for _, row in df_movies.iterrows():
            for gen in row["genres"]:
                cur.execute(query, (row["id"], gen))
        conn.commit()

In [25]:
df_countries = pd.read_csv("train/countries.csv")
df_countries.head()

,id,name
0,0,Франция
1,1,Мальта
2,5,Новая Зеландия
3,8,Куба
4,10,Пуэрто-Рико


In [26]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO countries (id, name) VALUES (%s, %s)"
        for _, row in df_countries.iterrows():
            cur.execute(query, (row["id"], row["name"]))
        conn.commit()

In [27]:
df_movies["countries"] = df_movies["countries"].map(ast.literal_eval)
df_movies["staff"] = df_movies["staff"].map(ast.literal_eval)

In [28]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO Movies_Countries (movie_id, country_id) VALUES (%s, %s)"
        for _, row in df_movies.iterrows():
            for item in row["countries"]:
                cur.execute(query, (row["id"], item))
        conn.commit()

In [29]:
df_staff = pd.read_csv("train/staff.csv")
df_staff.head()

,id,name,role
0,0,Юрий Волынцев,actor
1,1,Коннор Смит,actor
2,2,Майкл Гандольфини,actor
3,3,Майкл Маркс,producer
4,4,Кад Мерад,actor


In [30]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO staff (id, name, role) VALUES (%s, %s, %s)"
        for _, row in df_staff.iterrows():
            cur.execute(query, (row["id"], row["name"], row["role"]))
        conn.commit()

In [32]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO Movies_Staff (movie_id, staff_id) VALUES (%s, %s)"
        for _, row in df_movies.iterrows():
            for item in row["staff"]:
                cur.execute(query, (row["id"], item))
        conn.commit()

In [33]:
df_logs = pd.read_csv("train/logs.csv")
df_logs['datetime'] = pd.to_datetime(df_logs['datetime'], format='mixed')
df_logs.head()

,id,datetime,user_id,movie_id,duration
0,0,2023-04-06 15:00:00.071114+03:00,10250,427.0,485.0
1,1,2023-04-06 15:00:01.123928+03:00,2062,1521.0,129.0
2,2,2023-04-06 15:00:03.957246+03:00,12980,4598.0,2795.0
3,3,2023-04-06 15:00:04.990565+03:00,30646,5324.0,5094.0
4,4,2023-04-06 15:00:10.495017+03:00,43069,4291.0,75.0


In [37]:
df_users = pd.DataFrame({"id": df_logs["user_id"].unique()})
df_users.head()

,id
0,10250
1,2062
2,12980
3,30646
4,43069


In [41]:
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
password = pwd_context.hash("pass")
password

'$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSHmlHZAkjZhLC65G'

In [44]:
df_users["name"] = "user" + df_users["id"].astype(str)
df_users["email"] = df_users["name"] + "@example.com"
df_users["login"] = df_users["email"]
df_users["password"] = password
df_users.head()

,id,name,email,login,password
0,10250,user10250,user10250@example.com,user10250@example.com,$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSH...
1,2062,user2062,user2062@example.com,user2062@example.com,$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSH...
2,12980,user12980,user12980@example.com,user12980@example.com,$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSH...
3,30646,user30646,user30646@example.com,user30646@example.com,$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSH...
4,43069,user43069,user43069@example.com,user43069@example.com,$2b$12$gN/vtUyHXzDaNpILBd5sc.mcWRs4ucN5SzY3MSH...


In [46]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO users (id, name, email, login, password) VALUES (%s, %s, %s, %s, %s)"
        for _, row in df_users.iterrows():
            cur.execute(query, (row["id"], row["name"], row["email"], row["login"], row["password"]))
        conn.commit()

In [34]:
df_logs["movie_id"] = df_logs["movie_id"].astype(int)
df_logs["duration"] = df_logs["duration"].astype(int)
df_logs.head()

,id,datetime,user_id,movie_id,duration
0,0,2023-04-06 15:00:00.071114+03:00,10250,427,485
1,1,2023-04-06 15:00:01.123928+03:00,2062,1521,129
2,2,2023-04-06 15:00:03.957246+03:00,12980,4598,2795
3,3,2023-04-06 15:00:04.990565+03:00,30646,5324,5094
4,4,2023-04-06 15:00:10.495017+03:00,43069,4291,75


In [47]:
with conn:
    with conn.cursor() as cur:
        query = "INSERT INTO logs (id, user_id, datetime_, duration, movie_id) VALUES (%s, %s, %s, %s, %s)"
        for _, row in df_logs.iterrows():
            cur.execute(query, (row["id"], row["user_id"], row["datetime"], row["duration"], row["movie_id"]))
        conn.commit()

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port=5432
)

C:\Users\tryzh\AppData\Local\Temp\ipykernel_14596\8816142.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_movies = pd.read_sql("select * from movies;", conn)


,id,name,year,description
0,0,"Мама, я дома",2022-01-01,Где-то в глубинке вместе с дочерью и внуком жи...
1,1,Три метра над уровнем неба,2010-01-01,"История любви парня и девушки, принадлежащих к..."
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,"690 год нашей эры, Китай. Первая императрица д..."
3,3,Капитан,2017-01-01,Вторая мировая война подходит к концу. В это в...
4,4,Лазурный берег,2015-01-01,У бывшей танцовщицы Ванессы и писателя Роланда...


In [14]:
query = '''
select movies.id, movies.name, movies.year, movies.description, array_agg(genres.name) as movie_genres from movies
left join movies_genres on movies.id = movies_genres.movie_id
left join genres on movies_genres.genre_id = genres.id
group by movies.id;
'''

df_movies = pd.read_sql(query, conn)
df_movies.head()

,id,name,year,description,movie_genres
0,0,"Мама, я дома",2022-01-01,Где-то в глубинке вместе с дочерью и внуком жи...,[Драма]
1,1,Три метра над уровнем неба,2010-01-01,"История любви парня и девушки, принадлежащих к...","[Драма, Мелодрама, Боевик]"
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,"690 год нашей эры, Китай. Первая императрица д...","[Боевик, Фэнтези]"
3,3,Капитан,2017-01-01,Вторая мировая война подходит к концу. В это в...,"[Драма, Военный, Триллер, История]"
4,4,Лазурный берег,2015-01-01,У бывшей танцовщицы Ванессы и писателя Роланда...,"[Драма, Мелодрама]"


In [15]:
df_movies["movie_genres"] = df_movies["movie_genres"].apply(lambda x: [item if item else 'Отсутствуют' for item in x])  # noqa: F821
df_movies.head()

,id,name,year,description,movie_genres
0,0,"Мама, я дома",2022-01-01,Где-то в глубинке вместе с дочерью и внуком жи...,[Драма]
1,1,Три метра над уровнем неба,2010-01-01,"История любви парня и девушки, принадлежащих к...","[Драма, Мелодрама, Боевик]"
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,"690 год нашей эры, Китай. Первая императрица д...","[Боевик, Фэнтези]"
3,3,Капитан,2017-01-01,Вторая мировая война подходит к концу. В это в...,"[Драма, Военный, Триллер, История]"
4,4,Лазурный берег,2015-01-01,У бывшей танцовщицы Ванессы и писателя Роланда...,"[Драма, Мелодрама]"


In [16]:
df_movies['combined'] = df_movies.apply(
    lambda x: f"Название: {x['name']}. Жанры: {', '.join(x['movie_genres'])}. Год: {x['year']}. Описание: {x['description']}",
    axis=1
)
df_movies.head()

,id,name,year,description,movie_genres,combined
0,0,"Мама, я дома",2022-01-01,Где-то в глубинке вместе с дочерью и внуком жи...,[Драма],"Название: Мама, я дома. Жанры: Драма. Год: 202..."
1,1,Три метра над уровнем неба,2010-01-01,"История любви парня и девушки, принадлежащих к...","[Драма, Мелодрама, Боевик]",Название: Три метра над уровнем неба. Жанры: Д...
2,2,Детектив ди и тайна призрачного пламени,2010-01-01,"690 год нашей эры, Китай. Первая императрица д...","[Боевик, Фэнтези]",Название: Детектив ди и тайна призрачного плам...
3,3,Капитан,2017-01-01,Вторая мировая война подходит к концу. В это в...,"[Драма, Военный, Триллер, История]","Название: Капитан. Жанры: Драма, Военный, Трил..."
4,4,Лазурный берег,2015-01-01,У бывшей танцовщицы Ванессы и писателя Роланда...,"[Драма, Мелодрама]","Название: Лазурный берег. Жанры: Драма, Мелодр..."


In [18]:
df_movies = pd.read_parquet("backend/app/tasks/ml/preproccesed_movies.parquet")
df_movies.head()

,id,combined_features
0,0,"[-0.096071064, -0.2796514, -0.064225286, -0.08..."
1,1,"[-0.02544692, -0.21052195, -0.012258478, -0.11..."
2,2,"[-0.041481085, -0.23976798, -0.056806006, -0.0..."
3,3,"[-0.13428192, -0.30454487, -0.12308565, -0.108..."
4,4,"[-0.04161965, -0.23298249, -0.03673377, -0.107..."


In [19]:
embedding_size = df_movies["combined_features"][0].shape[0]
embedding_size

1536

In [20]:
query = "select user_id, datetime_ as \"datetime\", duration, movie_id from logs"

df_logs = pd.read_sql(query, conn)
df_logs.head()

,user_id,datetime,duration,movie_id
0,10250,2023-04-06 12:00:00.071114,485,427
1,2062,2023-04-06 12:00:01.123928,129,1521
2,12980,2023-04-06 12:00:03.957246,2795,4598
3,30646,2023-04-06 12:00:04.990565,5094,5324
4,43069,2023-04-06 12:00:10.495017,75,4291


In [22]:
df_logs['datetime'] = pd.to_datetime(df_logs['datetime'])
df_logs.sort_values(by=['user_id', 'datetime'], ascending=[True, True], inplace=True)
df_logs = df_logs.groupby(by=["user_id", "movie_id"])["datetime"].last().reset_index()

In [23]:
df_logs = df_logs.merge(df_movies, left_on="movie_id", right_on="id")[["datetime", "user_id", "combined_features", "movie_id"]]
df_logs.head()

,datetime,user_id,combined_features,movie_id
0,2023-06-12 15:28:52.927833,0,"[-0.029283525, -0.21942252, -0.032804504, -0.0...",12
1,2023-06-14 09:16:18.451369,0,"[-0.055958588, -0.24611095, -0.041972484, -0.1...",74
2,2023-06-14 15:47:34.563849,0,"[0.12725686, 0.08268017, 0.16072595, -0.115398...",107
3,2023-04-17 23:15:32.383760,0,"[-0.12217746, -0.27598616, -0.09268197, -0.118...",126
4,2023-04-18 08:59:47.414031,0,"[-0.121347286, -0.3023291, -0.10195325, -0.085...",165


In [26]:
df_logs.sort_values(by=['user_id', 'datetime'], ascending=[True, False], inplace=True)
dataset = df_logs.groupby("user_id").agg({
    "combined_features": lambda x: x[:20].tolist()[::-1],
    "movie_id": lambda x: x[:20].tolist()[::-1]
}).reset_index()
dataset.head()

,user_id,combined_features,movie_id
0,0,"[[-0.0072850958, -0.21902491, -0.02758896, -0....","[6830, 1576, 2965, 6746, 4207, 5893, 6151, 592..."
1,1,"[[-0.026403505, -0.21010612, -0.046826605, -0....","[5293, 3415, 7110, 3464, 611, 3135, 5774, 597,..."
2,2,"[[-0.0880633, -0.25398585, -0.058840286, -0.09...","[215, 3896, 4207, 107, 1909, 7308, 4868, 5542,..."
3,3,"[[-0.025762089, -0.22312973, -0.03183675, -0.1...","[4340, 6911, 3502, 6167, 7403, 5179, 1784, 741..."
4,4,"[[-0.05324371, -0.22430173, -0.0295939, -0.096...","[405, 2878, 4627, 4829, 6064, 4197, 2280, 4812..."


In [27]:
dataset = dataset[dataset["combined_features"].apply(lambda x: len(x) == 20)]

In [29]:
from backend.app.tasks.ml.RecSysModel import RecSysModel
import torch

PATH = "backend/app/tasks/ml/model.pth"

model = RecSysModel(embedding_size)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model.to(torch.device("cpu"))
model.eval()

RecSysModel(
  (embedding): Embedding(1536, 100)
  (act_func): ReLU()
  (fc1): Linear(in_features=3072, out_features=1536, bias=True)
  (fc2): Linear(in_features=1536, out_features=768, bias=True)
  (fc3): Linear(in_features=768, out_features=2, bias=True)
  (sm): Softmax(dim=-1)
)

In [30]:
for index, row in dataset.iterrows():
    if index < 100:
        continue
    pred, weights, ids = model.predict(row["combined_features"], df_movies)
    break

In [31]:
pred

[[0.03973257541656494, 0.9602674245834351],
 [0.04556479677557945, 0.9544351696968079],
 [0.053616732358932495, 0.9463832974433899],
 [0.057806920260190964, 0.9421930909156799],
 [0.060071222484111786, 0.9399288296699524],
 [0.06948409974575043, 0.9305158853530884],
 [0.07051938772201538, 0.9294806122779846],
 [0.071111761033535, 0.9288882613182068],
 [0.07142849266529083, 0.9285714626312256],
 [0.07364986091852188, 0.9263501167297363]]

In [32]:
ids

[2667, 3592, 484, 1978, 2817, 1976, 3996, 4543, 4913, 3512]

In [33]:
weights

[[0.05459457263350487,
  0.04972385987639427,
  0.053885605186223984,
  0.05118409916758537,
  0.04939917102456093,
  0.05511929467320442,
  0.0497223436832428,
  0.03692067041993141,
  0.04838993400335312,
  0.04976142942905426,
  0.052876584231853485,
  0.047605156898498535,
  0.04961233586072922,
  0.05498027428984642,
  0.04949083924293518,
  0.047243837267160416,
  0.05080661177635193,
  0.05229231342673302,
  0.050134044140577316,
  0.046257078647613525],
 [0.05177515000104904,
  0.049821339547634125,
  0.0530078150331974,
  0.05106058344244957,
  0.04762931913137436,
  0.05474786460399628,
  0.04832228645682335,
  0.03525133058428764,
  0.04887816309928894,
  0.05242713913321495,
  0.049200765788555145,
  0.04991423711180687,
  0.050612665712833405,
  0.05335128679871559,
  0.050468359142541885,
  0.049672674387693405,
  0.052781470119953156,
  0.05315180495381355,
  0.051100775599479675,
  0.04682491347193718],
 [0.05270479992032051,
  0.04804953560233116,
  0.05586722493171692

In [38]:
import numpy as np

np.argpartition(weights[0], -3)[-3:], weights[0]

(array([ 0, 13,  5], dtype=int64),
 [0.05459457263350487,
  0.04972385987639427,
  0.053885605186223984,
  0.05118409916758537,
  0.04939917102456093,
  0.05511929467320442,
  0.0497223436832428,
  0.03692067041993141,
  0.04838993400335312,
  0.04976142942905426,
  0.052876584231853485,
  0.047605156898498535,
  0.04961233586072922,
  0.05498027428984642,
  0.04949083924293518,
  0.047243837267160416,
  0.05080661177635193,
  0.05229231342673302,
  0.050134044140577316,
  0.046257078647613525])

In [40]:
tmp = dataset.loc[100, "movie_id"]
for w in weights:
    sub_recs = [tmp[idx] for idx in np.argpartition(w, -3)[-3:]]
    break
sub_recs

[6915, 2883, 5582]